In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
DATA_DIR = '../input/open-problems-multimodal'
%ls $DATA_DIR

evaluation_ids.csv                     test_multi_inputs.h5
metadata.csv                           train_cite_inputs.h5
metadata_cite_day_2_donor_27678.csv    train_cite_targets.h5
sample_submission.csv                  train_multi_inputs.h5
test_cite_inputs.h5                    train_multi_targets.h5
test_cite_inputs_day_2_donor_27678.h5


In [4]:
# train_cite_100 = pd.read_hdf(f'{DATA_DIR}/train_cite_inputs.h5',
#                              stop=100)
# print(train_cite_100.shape)
# train_cite_100.head()

In [5]:
# train_cite_targets_100 = pd.read_hdf(f'{DATA_DIR}/train_cite_targets.h5',
#                                      stop=100)
# print(train_cite_targets_100.shape)
# train_cite_targets_100.head()

In [6]:
from sklearn.neighbors import KNeighborsRegressor

In [7]:
# stop = 100
# train_X = pd.read_hdf(f'{DATA_DIR}/train_cite_inputs.h5',
#                       stop=stop)
# train_y = pd.read_hdf(f'{DATA_DIR}/train_cite_targets.h5',
#                       stop=stop)

In [8]:
train_X = pd.read_hdf(f'{DATA_DIR}/train_cite_inputs.h5')
train_y = pd.read_hdf(f'{DATA_DIR}/train_cite_targets.h5')

In [9]:
print('Start fitting...')
neigh = KNeighborsRegressor(n_neighbors=9)
neigh.fit(train_X, train_y) 
preds = pd.DataFrame(neigh.predict(train_X))
print('Preds done.')

del train_X
gc.collect()

Start fitting...
Preds done.


17

In [10]:
preds

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.501590,0.592784,0.926268,7.035001,6.863322,10.758928,9.520944,0.120384,0.969066,0.751912,...,0.459036,9.137736,0.809656,0.634390,0.072032,0.952109,3.839581,1.127186,2.926292,3.679075
1,0.958790,1.405730,1.334131,7.441905,6.999786,8.817806,9.467522,0.373423,0.837616,0.540694,...,0.369932,7.832520,0.433625,0.744496,0.394710,0.688417,3.402646,1.203001,3.080286,3.996535
2,0.417785,-0.050910,1.252796,4.462741,1.756896,7.131264,0.589949,0.780313,1.271102,0.016522,...,-0.445187,10.229585,1.991204,-0.075326,0.149838,0.929843,11.078556,0.154774,8.644727,3.692451
3,0.330948,-0.127473,0.832534,8.020531,6.640536,7.939573,17.675957,0.319240,0.700787,0.286209,...,0.254778,12.946860,0.522995,0.352635,-0.320252,0.835332,4.036196,1.833424,2.905849,3.375916
4,-0.248242,0.231249,0.757520,4.434628,3.497111,3.299439,-0.464640,0.304140,1.273222,0.413587,...,0.162029,7.030951,1.453715,-0.538925,-0.097509,0.718961,5.416647,-0.189498,8.569936,2.987624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70983,0.690879,0.499906,1.105918,5.842779,5.078206,8.485558,4.777509,-0.015348,0.715943,0.988136,...,0.150863,5.349822,0.539604,0.542639,0.142409,0.449744,4.009626,0.241022,2.099614,2.747022
70984,0.379059,1.277066,0.966305,5.780710,3.506581,6.215055,0.942005,0.345512,1.580637,0.216365,...,-0.212659,10.831044,2.410206,-0.096162,1.498787,0.728317,9.377787,-0.175441,8.788961,4.502172
70985,1.102599,0.746885,1.116761,4.744185,2.011682,5.748077,-0.067041,1.013290,1.317243,0.547468,...,-0.199638,8.007670,1.629465,0.107766,-0.277394,0.721376,8.859822,0.522583,8.492593,4.363985
70986,0.568294,0.157402,0.385556,6.717841,4.812807,10.070558,7.745582,0.287320,0.327400,0.487685,...,0.617756,12.563210,1.263593,0.255341,0.127508,0.883548,4.665799,1.398921,3.578389,3.719772


In [11]:
from scipy.stats import pearsonr

In [12]:
lhs = np.array(np.matrix(train_y).flatten())[0]
rhs = np.array(np.matrix(preds).flatten())[0]
print('baseline:', pearsonr(lhs, rhs)[0])

baseline: 0.8626628169461823
